In [2]:
import pandas as pd
import numpy as np

In [72]:
import joblib
def test_load_dfs(df_1,df_2):
    template_1 = joblib.load('tests/loans_template.pkl').dtypes
    template_3 = joblib.load('tests/recharges_template.pkl').dtypes
    assert (df_1.dtypes == template_1).all()
    assert (df_2.dtypes == template_3).all()

    return df_1,df_2


def load_dfs(paths=[
    'Brazil_DS_loans_2019-11-10_2019-12-05.csv',
    'Brazil_DS_prev_loans.csv',
    'Brazil_DS_recharges_2019-08-10_2019-12-05.csv']):
    
    loans_actual = pd.read_csv(paths[0],date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),parse_dates=['created_at','paid_at'])
    loans_prev = pd.read_csv(paths[1],date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),parse_dates=['created_at','paid_at'])
    loans = pd.concat([loans_actual,loans_prev],ignore_index=True)
    recharges = pd.read_csv(paths[2],date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),parse_dates=['recharge_timestamp'])
    loans['target'] = loans['paid_at'].isnull().apply(lambda x: 1 if x is True else 0)
    #return  test_load_dfs(loans, recharges)
    return  loans, recharges


loans, recharges = load_dfs()

# Exploratory data analysis

É interessante observar que os emprestimos não pagos não varia muito da mediana populacional, 5, entretanto utilizar essa informação pode gerar bias no modelo preditivo.


LEMBRETES:
    Criar intervalos de loans permitidos para cada usuário

In [93]:
recharges.query('uuid == "f0c50337ac0d4c09ae49b5e9064c4f72"')

,uuid,recharge_timestamp,recharge_value,balance_after_recharge,institution_name,institution_type
23,f0c50337ac0d4c09ae49b5e9064c4f72,2019-08-10 12:42:59,10.0,10.18,RV 31 NAC,Distribuidor Nacional
219,f0c50337ac0d4c09ae49b5e9064c4f72,2019-08-11 09:33:43,20.0,20.22,RV 31 NAC,Distribuidor Nacional
327,f0c50337ac0d4c09ae49b5e9064c4f72,2019-08-10 23:10:21,5.0,5.19,NaN,NaN
972,f0c50337ac0d4c09ae49b5e9064c4f72,2019-08-12 07:12:00,5.0,5.00,NaN,NaN
1538,f0c50337ac0d4c09ae49b5e9064c4f72,2019-08-13 11:15:41,5.0,5.56,NaN,NaN
...,...,...,...,...,...,...
90597,f0c50337ac0d4c09ae49b5e9064c4f72,2019-11-29 10:53:00,5.0,5.04,NaN,NaN
91234,f0c50337ac0d4c09ae49b5e9064c4f72,2019-12-04 00:17:51,10.0,13.01,NaN,NaN
91994,f0c50337ac0d4c09ae49b5e9064c4f72,2019-12-02 02:25:16,5.0,8.52,NaN,NaN
94155,f0c50337ac0d4c09ae49b5e9064c4f72,2019-12-02 12:03:51,20.0,20.00,RV 31 NAC,Distribuidor Nacional


In [91]:
recharges.groupby('uuid').count().sort_values(by='recharge_timestamp', ascending=False)

,recharge_timestamp,recharge_value,balance_after_recharge,institution_name,institution_type
uuid,,,,,
f0c50337ac0d4c09ae49b5e9064c4f72,134,134,134,70,70
613ab91c2d744db593a6e8ee075f7ef8,104,104,104,58,58
3007a812fc2345fb94772b6cb63e4313,102,102,102,69,69
00c1526f1c984ab2b56c9d3ad5dc5227,86,86,86,59,59
1af37ef8003d4878a8dcf1c6fcba8e31,85,85,85,44,44
...,...,...,...,...,...
654db83d9d1d46fbb10bb5fb25096b5e,1,1,1,0,0
ad0ceeb8dd3544b38b184419ce7eb032,1,1,1,0,0
e8d34a12efab4aa6b64be96c5606d642,1,1,1,0,0


In [104]:
dates_interval = ['20151101','20201231']


t_loans = loans.query(f'{dates_interval[0]} <= created_at <= {dates_interval[1]}').copy()
t_recharges = recharges.query(f'{dates_interval[0]} <= recharge_timestamp <= {dates_interval[1]}').copy()

t_recharges['delta_after_recharge'] = t_recharges['balance_after_recharge'] - t_recharges['recharge_value']


print('General Loans stats \n',t_loans['amount'].describe(),'\n',f"Median : {t_loans['amount'].median()}")
print('Paid stats \n',t_loans.query('target == 0').groupby('uuid').sum('amount').sort_values('amount',ascending=False)['amount'].describe())
print('Not Paid stats \n',t_loans.query('target == 1').groupby('uuid').sum('amount').sort_values('amount',ascending=False)['amount'].describe())

#t_loans
sum_amount_p = t_loans.groupby('uuid').sum()['amount'] #within INTERVAL_LENGTH days ago sum of amount of loans paid
count_p = t_loans.groupby('uuid').count()['amount']#within INTERVAL_LENGTH days ago count of loans paid

#t_recharges
freq_recharge = t_recharges.groupby('uuid').count()['recharge_timestamp']
sum_recharge_value = t_recharges.groupby('uuid').sum()['recharge_value']

General Loans stats 
 count    18961.000000
mean         5.551326
std          1.566274
min          3.850000
25%          5.000000
50%          5.000000
75%          5.000000
max         10.000000
Name: amount, dtype: float64 
 Median : 5.0
Paid stats 
 count    7530.000000
mean       12.283533
std        17.312300
min         5.000000
25%         5.000000
50%         5.000000
75%        10.000000
max       265.000000
Name: amount, dtype: float64
Not Paid stats 
 count    2469.000000
mean        5.169583
std         0.931548
min         5.000000
25%         5.000000
50%         5.000000
75%         5.000000
max        18.700000
Name: amount, dtype: float64
